<a href="https://colab.research.google.com/github/nan-park/cp1_project/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Import**

In [71]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import csv

In [279]:
# (체크) 데이터 불러오는 것도 나중에 함수화 하기
def data_load():
  from google.colab import drive  # (체크) 이 라이브러리도 쓰면 안 되는지 질문하기(!wget 써야 하는지)
  drive.mount('/content/drive')
  df = pd.read_csv('/content/drive/MyDrive/binary_dataset.csv')
  return df
df = data_load()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [73]:
df.head()

,x1,x2,x3,x4,x5,x6,x7,x8,y
0,136.093750,51.691005,-0.045909,-0.271816,9.342809,38.096400,4.345438,18.673649,0
1,99.367188,41.572202,1.547197,4.154106,27.555184,61.719016,2.208808,3.662680,1
2,100.890625,51.890394,0.627487,-0.026498,3.883779,23.045267,6.953168,52.279440,0
3,120.554688,45.549905,0.282924,0.419909,1.358696,13.079034,13.312141,212.597029,1
4,121.882812,53.042675,0.200521,-0.282219,2.116221,16.580876,8.947603,91.011762,0


In [243]:
# 학습/테스트 데이터 뒤섞기
def df_shuffle(df):
  return df.sample(frac=1).reset_index(drop=True)

# 특성, 타겟 나누기
def divide_xy(df):
  target = 'y'
  features = list(df.columns)
  features.remove(target)

  X = df[features]
  y = np.array(df[target]).reshape(-1, 1)
  return X, y

# 학습/테스트 데이터 분리하기
def train_test_divide(X, y, test_size=0.2):  # X: pandas dataframe, y: pandas series
  length = len(y)
  test_index = int(length * test_size)

  X_test = X[:test_index]
  y_test = y[:test_index]

  X_train = X[test_index:]  
  y_train = y[test_index:]

  return X_train, y_train, X_test, y_test

In [244]:
# 위의 함수 통합
def train_test_split(df, shuffle=True, test_size=0.2):
  if shuffle:
    df = df_shuffle(df)
  
  X, y = divide_xy(df)
  return train_test_divide(X, y, test_size=test_size)

In [245]:
X_train, y_train, X_test, y_test = train_test_split(df)

In [246]:
# 미니배치 설정
def set_mini_batch(X, y): # train, test 들어올 예정
  # 4개씩 미니배치 설정. 나머지는 버리기
  length = len(y)
  num = length // 4 # 미니배치 개수
  X_batch_list = []
  y_batch_list = []
  for i in range(num):
    i = i * 4
    # 비복원 추출. 데이터가 적기 때문에 겹치지 않는 게 나을 듯.
    X_batch_list.append(X[i:i+4]) # index: 0~4, 4~8, 8~12, ...
    y_batch_list.append(y[i:i+4])
  return X_batch_list, y_batch_list

X_batch_list, y_batch_list = set_mini_batch(X_train, y_train)

In [247]:
# 층의 노드 개수(n), 미니배치 하나 개수(4개) -> 가중치 4xn

# Xavier 초기화(활성화함수가 시그모이드일 때 잘 동작), 편향 포함한 후에 분리하기
# 이전 층 노드 개수가 n, 현재 층 노드 개수 m일 때, 표준편차가 2/루트(n+m)인 정규분포로 초기화

def initialize_parameter(n, m):
  init = np.random.normal(0, 2/((n+m)**2), (n+1, m))
  W = init[:-1, :]
  b = init[-1, :]
  return W, b

initialize_parameter(4, 5)

(array([[-0.04399682,  0.02702013, -0.00144542, -0.05405016,  0.00888407],
        [ 0.02606975, -0.0024731 ,  0.00364449,  0.02222029,  0.00930477],
        [-0.00820177,  0.00110716,  0.02111473,  0.0208526 ,  0.03390977],
        [-0.00134601,  0.00263913,  0.01704033,  0.02435149,  0.00880125]]),
 array([ 0.00024967,  0.03186318,  0.02165979, -0.01878162,  0.00786483]))

In [248]:
def sigmoid(x):
    return 1 / (1 +np.exp(-x))

def classification(x):
  if x < 0.5:
    return 0
  else:
    return 1

In [260]:
class Layer():  # 은닉층, 출력층
  def __init__(self, node_num, activation='linear'):
    self.node_num = node_num
    self.activation = activation
    self.prev = None
    self.next = None

  def set_weights(self):
    if self.prev is not None:
      prev_node_num = self.prev.node_num
      self.W, self.b = initialize_parameter(prev_node_num, self.node_num)
  
  @property
  def X(self):
    return self._X
  @X.setter
  def X(self, value):
    self._X = value

class Dense(Layer):
  def __init__(self, node_num, activation='linear'):
    # super().__init__(self, node_num)
    self.node_num = node_num
    self.activation = activation
    self.prev = None
    self.next = None
  
  def output(self):
    answer = np.dot(self._X, self.W) + self.b
    if self.activation == 'linear':
      return answer
    elif self.activation == 'sigmoid':
      # answer = sigmoid(answer)
      # classify = np.vectorize(classification)
      # return classify(answer)
      return answer

class Input(Layer): # prev가 없음
  def __init__(self, node_num, activation='linear'):
    self.node_num = node_num
    self.activation = activation
    self.prev = None
    self.next = None

  def output(self):
    return self._X

In [261]:
# Sequential([])에 layer 쌓고 서로 연결되도록 하기. 가중치 초기화 가능해야 함
class Sequential():
  def __init__(self, layer_list):
    if len(layer_list)==0:
      self.head = None
      self.tail = None
    elif len(layer_list)==1:
      self.head = layer_list[0]
      self.tail = layer_list[0]
    else:
      self.head = layer_list[0]
      iterator = self.head
      for layer in layer_list[1:-1]:
        layer.prev= iterator
        iterator.next = layer
        iterator = layer
      iterator.next = layer_list[-1]
      self.tail = layer_list[-1]
      self.tail.prev = iterator

    # 가중치, 편향 초기화
    iterator = self.head
    while iterator:
      iterator.set_weights()
      iterator = iterator.next

  @property
  def input(self):
    return self._input
  @input.setter
  def input(self, value):
    self._input = value
    self.head.X = value

  def output(self):
    iterator = self.head  # Input
    while iterator.next:
      iterator.next.X = iterator.output()
      iterator = iterator.next
    return iterator.output()
    

In [251]:
model = Sequential([Input(8), Dense(16), Dense(32), Dense(1, activation='sigmoid')])

In [252]:
# 미니배치 하나(4개) 데이터 넣어서 output 내기
model.input = X_batch_list[0] # (4, 8) * (8, 16) -> (4, 16)
y_predict = model.output()
y_predict

array([[0],
       [0],
       [0],
       [0]])

In [253]:
# 미니배치 하나의 정확도 산출하기
y_batch_list[0]

array([[0],
       [0],
       [0],
       [0]])

In [273]:
def evaluate(X_batch_list, y_batch_list, model):
  def accuracy(y_pred, y_batch):
    return sum(y_pred == y_batch)[0] / y_pred.shape[0]

  def cross_entropy(y_pred_prob, y_batch):
    delta = 1e-7
    return -np.sum(y_batch * np.log(y_pred_prob + delta)) / y_batch.shape[0]

  accuracy_list = []
  cross_entropy_list = []
  for i in range(len(X_batch_list)):
    X_batch = X_batch_list[i]
    y_batch = y_batch_list[i]
    model.input = X_batch
    y_pred_prob = model.output()
    classify = np.vectorize(classification)
    y_pred = classify(y_pred_prob)

    accuracy_list.append(accuracy(y_pred, y_batch))
    cross_entropy_list.append(cross_entropy(y_pred_prob, y_batch))

  accuracy = np.mean(accuracy_list)
  cross_entropy = np.mean(cross_entropy_list)
  return accuracy, cross_entropy


In [277]:
accuracy, cross_entropy = evaluate(X_batch_list, y_batch_list, model)
print(f"[Epoch 1] TrainData - Loss = {round(cross_entropy, 3)}, Accuracy = {round(accuracy, 3)}")

[Epoch 1] TrainData - Loss = 4.03, Accuracy = 0.75
